# Prepare environment

In [ ]:
#Import relevant packages
import numpy as np
import pandas as pd
from matplotlib import rcParams
import os
import scanpy as sc

import matplotlib as mpl
import matplotlib.pyplot as plt

#For nice color schemes
import cmocean

#For barplots
import seaborn as sns

In [ ]:
#Import scVI
import scvi
from scvi.model.utils import mde

scvi.settings.verbosity = 40

In [ ]:
#Set fontsize
plt.rcParams.update({'font.size': 20})

# Read in datasets

For example, to read in filtered feature matrices from GEO:

In [ ]:
#Set wd
os.chdir('/hpc/group/goldsteinlab/Python/GEO_accession/Lee_et_al_Nat_Comm_GBM_TILs')

In [ ]:
#Read in 10x Cell Ranger output counts matrix 
LB3612T = sc.read_10x_mtx('ndGBM_outs/LB3612T/', var_names='gene_symbols', cache=True)

#Add metadata
LB3612T.obs['batch1'] = 'LB3612T'
LB3612T.obs['orig_patient']='LB3612'
LB3612T.obs['tumor'] = 'ndGBM'
LB3612T.obs['study'] = 'Lee_et_al_2021'

In [ ]:
#Read in 10x Cell Ranger output counts matrix 
LB3771T = sc.read_10x_mtx('ndGBM_outs/LB3771T/', var_names='gene_symbols', cache=True)

#Add metadata
LB3771T.obs['batch1'] = 'LB3771T'
LB3771T.obs['orig_patient']='LB3771'
LB3771T.obs['tumor'] = 'ndGBM'
LB3771T.obs['study'] = 'Lee_et_al_2021'

Continue reading in all desired datasets, adding appropriate obs

# Concatenate objects

In [ ]:
#Concatenate anndata objects
#These are all composed of single filtered feature matrices from data read in, as above
adata = LB3612T.concatenate([LB3771T, normal_brain_samples, GBM_samples,
                            brain_met_samples, PBMC_samples], index_unique=None, join="outer")

In [ ]:
#Calculate QC statistics
adata.var['mito'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mito'], percent_top=None, log1p=False, inplace=True)

In [ ]:
#Plot
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mito'],
             jitter=0.4, multi_panel=True)
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mito')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
#QC Filter data by slicing anndata object
adata = adata[adata.obs.n_genes_by_counts < 8000, :]
adata = adata[adata.obs.total_counts > 500, :]
adata = adata[adata.obs.pct_counts_mito < 25, :]

In [ ]:
#Prep for HVG and scvi
#log1p the data
adata.obs["log1p_total_counts"] = np.log1p(adata.obs["total_counts"])

#Create layers
adata.layers["counts"] = adata.X.copy()
adata.layers['norm'] = adata.X.copy(); sc.pp.normalize_total(adata, target_sum=1e4, layer="norm")

# hvg selection

In [ ]:
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=5000,
    subset=False,
    layer="counts",
    flavor="seurat_v3"
)

In [ ]:
adata.var['mean_'] = np.array(adata.X.mean(0))[0]
adata.var['frac_zero'] = 1 - np.array((adata.X > 0).sum(0))[0] / adata.shape[0]

fig, ax = plt.subplots(figsize=(9,6))
ax.scatter(adata.var.mean_, adata.var.frac_zero, s=1)
ax.set_xscale("log")

In [ ]:
#Calculate Poisson gene selection
df_poisson = scvi.data.poisson_gene_selection(
    adata, n_top_genes=5000, batch_key="batch1", inplace=False
)

df_poisson[df_poisson.highly_variable].sort_values('prob_zero_enrichment_rank')

pd.crosstab(df_poisson.highly_variable, adata.var.highly_variable)

is_hvg = df_poisson.highly_variable

adata.varm['df_poisson']= df_poisson

adata_query = adata[:, is_hvg].copy()
print(adata_query)

# Set up and run scvi

In [ ]:
#Set up scvi model

#Can insert batch_key here if desired
scvi.model.SCVI.setup_anndata(
    adata_query,
    layer="counts",
    categorical_covariate_keys=["orig_patient", 'tumor'],
    continuous_covariate_keys=["pct_counts_mito"],
    batch_key='batch1'
)

model = scvi.model.SCVI(adata_query, gene_likelihood="nb")

model.view_anndata_setup()

In [ ]:
#Train model

#Training parameters
train_kwargs = dict(
    early_stopping=True,
    early_stopping_patience=20,
    enable_model_summary=True,
    enable_progress_bar=True,
    enable_checkpointing=True,
    max_epochs=800
)

#Be sure GPU is enabled to run this
model.train(**train_kwargs)

In [ ]:
#Plot model results
train_elbo = model.history['elbo_train'][1:]
test_elbo = model.history['elbo_validation']

ax = train_elbo.plot()
test_elbo.plot(ax = ax)

In [ ]:
#Fit model to data

#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()

adata.obsm["X_scVI_1.1"] = latent

#Calculate neighbors using scVI model input
sc.pp.neighbors(adata, use_rep="X_scVI_1.1")
sc.tl.umap(adata, min_dist=0.5)

In [ ]:
#Run leiden clustering based on neighbors
sc.tl.leiden(adata, key_added="leiden_scVI_1.1", resolution=2.0)

# Assess quality

In [ ]:
#QC UMAPs
sc.pl.umap(
    adata,
    color=["n_genes_by_counts", "total_counts", "pct_counts_mito", "log1p_total_counts"],
    cmap="cubehelix_r",
    s=3,
    ncols=2,
)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
sc.pl.umap(adata, color="tumor", cmap="cmo.matter", s=3, ax=ax, vmax="p99.99")
fig, ax = plt.subplots(figsize=(12, 8))
sc.pl.umap(adata, color="leiden_scVI_1.1", legend_loc="on data", ax=ax, s=3)
fig, ax = plt.subplots(figsize=(12, 8))
sc.pl.umap(adata, color="orig_patient", legend_loc="right margin", ax=ax, s=3)

In [ ]:
#Additional QC bar graphs
adata_query.obs['cluster'] = adata.obs["leiden_scVI_1.1"].copy()

#Plot Log1p total counts
fig, ax = plt.subplots(figsize=(30,6))
sns.boxenplot(data=adata_query.obs, x="cluster", y="log1p_total_counts", ax=ax)

#Plot Pct counts mito
fig, ax = plt.subplots(figsize=(30,6))
sns.boxenplot(data=adata_query.obs, x="cluster", y="pct_counts_mito", ax=ax)

In [ ]:
#Assess gene expression across featureplots on the model embedding
#GBM specific
genes = ['NRG3', 'NCAM1', 'MBP',
         'SOX2', 'OLIG2', 'GFAP',
         'LUM', 'VWF', 'MKI67']

sc.pl.umap(
    adata,
    color=genes,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.matter",
    ncols=3,
    frameon=False,
    vmax="p99.5",
    layer="norm",
    save=False
)

In [ ]:
#Neurons
genes = ['PLXDC2', 'ELMO1', 'FRMD4A',
        'LDLRAD4', 'SRGAP2', 'FKBP5',
        'DOCK4', 'CELF2', 'NCAM1']

sc.pl.umap(
    adata,
    color=genes,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.matter",
    ncols=3,
    frameon=False,
    vmax="p99.5",
    layer="norm",
    save=False
)

In [ ]:
#Astrocytes
genes = ['GFAP', 'S100B', 'ALDH1L1']

sc.pl.umap(
    adata,
    color=genes,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.matter",
    ncols=3,
    frameon=False,
    vmax="p99.5",
    layer="norm",
    save=False
)

In [ ]:
#Breast and immune
genes = ['EPCAM', 'PDGFRA', 'PECAM1',
        'MCAM', 'PDGFRB', 'MKI67']

sc.pl.umap(
    adata,
    color=genes,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.matter",
    ncols=3,
    frameon=False,
    vmax="p99.5",
    layer="norm",
    save=False
)

In [ ]:
#More detailed immune
genes = ['PTPRC', 'CD3D', 'CD68',
        'CD8A', 'CD4', 'IL7R', #second two are CD4 markers
        'FGFBP2', 'FCGR3A', 'CX3CR1', #NK cells
         'CD19', 'CD79A', 'MS4A1', #B cell specific
        'IGHG1', 'MZB1', 'SDC1', #Plasma cell specific
        'CD14', 'S100A12', 'CLEC10A', #monocyte specific
         'C1QA', 'C1QB', 'C1QC', #macrophage specific
         'CD1C', 'TPSB2', 'TPSAB1'] #first is DC, last 2 are Mast cells 
sc.pl.umap(
    adata,
    color=genes,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.matter",
    ncols=3,
    frameon=False,
    vmax="p99.5",
    layer="norm",
    save=False
)

In [ ]:
#Microglia
genes = ['CX3CR1', 'TMEM119', 'CD68', 'PTPRC']

sc.pl.umap(
    adata,
    color=genes,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.matter",
    ncols=3,
    frameon=False,
    vmax="p99.5",
    layer="norm",
    save=False
)

In [ ]:
#Find cluster markers
sc.tl.rank_genes_groups(adata, 'leiden_scVI_1.1', method='wilcoxon', layer='norm', use_raw=False)
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(30)

From here, assess low quality cells and doublet clusters from graphs and data produced above

In [ ]:
# If any bad quality clusters present, remove
bad_clust=['51', '34']

#Filter out bad clusters
to_keep=(~adata.obs['leiden_scVI_1.1'].isin(bad_clust))

#Copy over to new anndata object
adata = adata[to_keep].copy()

Now iteratively run through another round of scvi model training, starting with hvg selection step. Continue until few/no low quality cell clusters remain

# Annotate cell clusters

In [ ]:
#Rename scVI clusters with dash (need to do this for follow-up naming with dictionary to work properly)
new_cluster_names = ['_0', '_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10',
                     '_11', '_12', '_13', '_14', '_15', '_16', '_17', '_18', '_19', '_20',
                    '_21', '_22', '_23', '_24', '_25', '_26', '_27', '_28', '_29', '_30', 
                     '_31', '_32', '_33', '_34', '_35', '_36', '_37', '_38', '_39', '_40',
                     '_41', '_42', '_43', '_44', '_45', '_46', '_47', '_48', '_49', '_50']
adata.rename_categories('leiden_scVI_1.1', new_cluster_names)

In [ ]:
#Begin annotating
#broad immune names

old_to_new = dict(
    _0='Microglia',
    _1='Lymphoid',
    _2='Tumor',
    _3='Myeloid',
    _4='Myeloid',
    _5='Tumor',
    _6='Oligodendrocytes',
    _7='Tumor',
    _8='Tumor',
    _9='Microglia',
    _10='Oligodendrocytes',
    _11='Tumor',
    _12='Microglia',
    _13='Tumor',
    _14='Lymphoid',
    _15='Neurons',
    _16='Lymphoid',
    _17='Myeloid',
    _18='Oligodendrocytes',
    _19='Myeloid',
    _20='Astrocytes',
    _21='Neurons',
    _22='Tumor',
    _23='Myeloid',
    _24='Oligodendrocytes',
    _25='Microglia',
    _26='Myeloid',
    _27='Myeloid',
    _28='Myeloid',
    _29='Fibroblasts',
    _30='Myeloid',
    _31='Lymphoid',
    _32='Tumor',
    _33='Oligodendrocytes',
    _34='Myeloid',
    _35='Tumor',
    _36='Tumor',
    _37='Neurons',
    _38='Microglia',
    _39='Lymphoid',
    _40='Neurons',
    _41='Tumor',
    _42='Lymphoid',
    _43='Vascular',
    _44='Myeloid',
    _45='Lymphoid',
    _46='Lymphoid',
    _47='Myeloid',
    _48='Tumor',
    _49='Tumor',
    _50='Myeloid'
)


adata.obs['cluster_names_broad'] = (
    adata.obs['leiden_scVI_1.1']
    .map(old_to_new)
    .astype('category')
)

In [ ]:
#Plot on embedding
fig, ax = plt.subplots(figsize=(12, 8))
sc.pl.umap(adata, color="cluster_names_broad", legend_loc="right_margin", ax=ax, s=1, palette=palette, frameon=False,
          save=False)

In [ ]:
genes=['PTPRC', 'CX3CR1', 'TMEM119', 'PLXDC2', 'FKBP5', 'DOCK4', 'FRMD4A', 'LDLRAD4', 'SRGAP2',
       'CD68', 'CD14', 'S100A12', 'C1QA', 'CD3D', 'CD8A', 'IL7R', 'LUM', 'PDGFRB', 
       'VWF', 'NCAM1', 'NRG3', 'GFAP', 'ALDH1L1', 'MBP', 'SOX2', 'OLIG2', 'S100B', 
       'EPCAM', 'PDGFRA', 'MKI67']

fig, ax = plt.subplots(figsize=(8,10))
sc.pl.dotplot(adata, genes, groupby='cluster_names_broad', layer='norm', ax=ax, cmap='Reds',
             swap_axes=True,
                         categories_order=['Microglia', 'Myeloid', 'Lymphoid', 
                                           'Fibroblasts', 'Vascular', 'Neurons', 'Astrocytes',
                                          'Oligodendrocytes', 'Tumor'],
             save=False)

In [ ]:
adata.write('glioma_normalbrain_TILs_brainmets_scvi.h5ad')